In [7]:
import torch as t
trg = t.tensor([[1,2,3],[4,5,6]])
print(trg[0,:])

tensor([1, 2, 3])


In [43]:
import os
import re
from collections import Counter
# 从少样本中找出高频词，作为这类文本的标识
def get_special_token_from_less_sample(dir_path,high_freq_file):    
    file_list= os.listdir(dir_path)
    all_words = [] # 少样本中出现的词汇
    for file in file_list:
        cur_file_path = os.path.join(dir_path,file)
        with open(cur_file_path, 'r') as f:
            for line in f:
                line = line.strip("\n")
                line = re.split(r"[，。！？ ]",line)
                line = [_ for _ in line if _ !=""]
                all_words.extend(line)
                # print(line)
#                 break
    word_cnt = dict(Counter(all_words))
    print(type(word_cnt))
    word_cnt = sorted(word_cnt.items(),key = lambda x:x[1],reverse=True)
    

    # 再去除高频词
    high_word_freq = {} # 读取高频词列表
    with open(high_freq_file, 'r') as f:        
        cont = json.load(f)
        # print(cont)
    
#     print(type(cont))
    cont = dict(cont)
    top_k = 0  # 取top_k 在200 为高频词
    for item in cont.items():
        key,value = item
        high_word_freq[key] =value
        if top_k > 500:
            break
        top_k+=1
#     print(high_word_freq)
    
    # 找出文档中的高频词
    top_k = 0
    for item in word_cnt:
        key,value = item
        if key not in high_word_freq.keys():
            print(item)
            top_k += 1
        if top_k > 20:
            break
get_special_token_from_less_sample("/home/lawson/program/daguan/risk_data_grand/data/5-24",
                                   '/home/lawson/program/daguan/risk_data_grand/data/small_json/word_freq.json')

<class 'dict'>
('8224', 24)
('648', 17)
('12742', 15)
('23349', 11)
('13343', 10)
('26526', 9)
('3382', 9)
('6288', 9)
('2910', 8)
('4004', 8)
('14489', 7)
('29640', 7)
('1826', 7)
('3848', 6)
('16157', 6)
('11259', 6)
('27762', 6)
('18785', 6)
('18244', 5)
('21853', 5)
('3240', 5)


In [32]:
import json
# 从unlable 文档中抽取出所有的
def get_all_title_from_unlabel():
    all_title= [] # 
    source = "/home/lawson/program/daguan/risk_data_grand/data/datagrand_2021_unlabeled_data.json"
    dest = "/home/lawson/program/daguan/risk_data_grand/data/unlabel_title.txt"
    with open(source,'r',errors='ignore') as f:
        line = f.readline()
        while line:
            line = json.loads(line,strict=False)
            # print(line)
            title = line['title']            
            all_title.append(title)
            line = f.readline()
    
    with open(dest,'w') as f:
        for line in all_title:
            f.write(line+"\n")
get_all_title_from_unlabel()

KeyboardInterrupt: 

In [ ]:
import torchtext as tt
import torchtext.data import TabularDataset
# 定义filed
datafields=[("context",TEXT),("label",LABEL)] ## TEXT field, LABEL 
fieldtest_field=[("context",TEXT),("label",LABEL)]

#
train_file,valid_file = TabularDataset.splits(path ="/home/lawson/program/wheels/seq2seq/data/en.xml" ,skip_header=True,fields=datafields)
print(train_file[1].context[:3])
print(train_file[1].label)

In [ ]:
from torchtext import data
from torchtext.vocab import Vectors
from torch.nn import init
from tqdm import tqdm

tokenize = lambda x: x.split()
# fix_length指定了每条文本的长度，截断补长
TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, fix_length=200)
LABEL = data.Field(sequential=False, use_vocab=False)

# 读取数据
train_data = pd.read_csv('data/train_one_label.csv')
valid_data = pd.read_csv('data/valid_one_label.csv')
test_data = pd.read_csv("data/test.csv")
TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True)
LABEL = data.Field(sequential=False, use_vocab=False)

# get_dataset构造并返回Dataset所需的examples和fields
def get_dataset(csv_data, text_field, label_field, test=False):
# id数据对训练在训练过程中没用，使用None指定其对应的field
    fields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("comment_text", text_field), ("toxic", label_field)]       
    examples = []

    if test:
        # 如果为测试集，则不加载label
        for text in tqdm(csv_data['comment_text']):
            examples.append(data.Example.fromlist([None, text, None], fields))
    else:
        for text, label in tqdm(zip(csv_data['comment_text'], csv_data['toxic'])):
            examples.append(data.Example.fromlist([None, text, label], fields))
    return examples, fields

# 得到构建Dataset所需的examples和fields
train_examples, train_fields = get_dataset(train_data, TEXT, LABEL)
valid_examples, valid_fields = get_dataset(valid_data, TEXT, LABEL)
test_examples, test_fields = get_dataset(test_data, TEXT, None, test=True)

# 构建Dataset数据集
train = data.Dataset(train_examples, train_fields)
valid = data.Dataset(valid_examples, valid_fields)
test = data.Dataset(test_examples, test_fields)
